# ニューラルネットワークによる2クラス分類

---
## 目的
多層パーセプトロン（Multi Layer Perceptron; MLP）を用いて，乳ガンデータの2クラス分類を行う．

## 対応するチャプター
* 6.2.2: ベルヌーイ分布出力のためのシグモイドユニット
* 8.1.3: バッチアルゴリズムとミニバッチアルゴリズム
* 8.3.1: 確率的勾配降下法

## モジュールのインポート
プログラムの実行に必要なモジュールをインポートします．
実験にはPythonの機械学習用ライブラリであるscikit-learnと深層学習ライブラリの一つであるchainerを使用します．
使用するクラス，関数は以下の通りです．

* `numpy`（説明は割愛）
* `load_breast_cancer`はデータセットを読み込むための関数
* `train_test_split`はデータを学習用とテスト用のデータに分割するための関数
* `chainer`は深層学習を使用するためのPythonライブラリ
* `chainer.cuda`はchainer上でGPUを使用した計算を行うためのモジュール

In [ ]:
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

import chainer
from chainer import Variable
import chainer.functions as F
import chainer.links as L

## データセットの読み込み
実験に使用するデータセットを読み込みます．

今回はscikit-learnが提供する`load_breast_cancer`関数を用いて，データを読み込みます．
breast cancer datasetは肺癌のデータセットであり，クラス数は悪性腫瘍 (malignant)と良性腫瘍 (benign) の2クラス，データ数は569（悪性腫瘍 (malignant): 220, 良性腫瘍 (benign): 357）のデータセットです．
各データは細胞核の半径や面積，テクスチャ情報を表現した30次元のベクトルデータです．

読み込んだ全てのデータのうち，`breast_cancer_data.data`でデータを読み込み，`breast_cancer_data.target`で教師ラベルをそれぞれ読み込みます．

In [ ]:
breast_cancer_data = load_breast_cancer()
x = breast_cancer_data.data.astype(np.float32)
y = breast_cancer_data.target.astype(np.int32)

print(x.shape)
print(y.shape)

## データの分割と正規化
上記で読み込んだデータを学習用データとテストデータに分割し，正規化を行います．

データの分割には`train_test_split`関数を使用します．
はじめに`test_sample_ratio`でテストに用いるデータ数の割合を指定します．
その後，`train_test_split`関数を用いて指定した割合でデータを分割します．
`random_state`はデータをランダムに分割する際のseedです．
seedを変更，または指定しないことで，無作為にデータを分割することが可能です．

次に正規化を行います．
データ$x$の最小値を$x_{min}$，最大値を$x_{max}$としたとき，次の式で正規化を行います．
$$x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}$$

`np.min`と`np.max`で学習データの最大，最小値を取得し，上記の式に従い0~1の範囲に値を正規化します．

In [ ]:
# テストデータの割合を指定
test_sample_ratio = 0.2

# データを分割
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_sample_ratio, random_state=0)

# データの正規化
x_min = np.min(x_train, axis=0)
x_max = np.max(x_train, axis=0)

x_train = (x_train[:, ] - x_min) / (x_max - x_min)
x_test = (x_test[:, ] - x_min) / (x_max - x_min)

## ネットワークモデルの定義
ニューラルネットワーク（多層パーセプトロン）を定義します．
ここでは，入力層，出力層から構成される2層の多層パーセプトロンを定義します．

入力層のユニット数は入力データのサイズによります．
ここではNoneとし，データにより変更できるようにしておきます．

中間層と出力層のユニット数は引数として与え，それぞれ`n_hidden`，`n_out`とします．
Chainerでは`__init__`関数にこれらの引数を与えて各層を定義します．
各層の定義には`Linear`関数を使用します．
これは全結合層を意味しています．

そして，`__call__`関数で，定義した層を接続して処理するように記述します．
`__call__`関数の引数`x`は入力データです．
それを`__init__`関数で定義した`l1`という層に与え，その出力を活性化関数である`tanh`関数に入力します．
その出力を`h`としています．
`h`はさらに`l2`層に入力され，最終的な結果を出力します．

In [ ]:
class MLP(chainer.Chain):
    def __init__(self, n_hidden, n_out):
        super(MLP, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_hidden)
            self.l2 = L.Linear(n_hidden, n_out)

    def forward(self, x):
        h = F.tanh(self.l1(x))
        h = self.l2(h)
        return h

## ネットワークの作成と学習の準備
上のプログラムで定義したネットワークを作成します．


まず，中間層と出力層のユニット数を定義します．
ここでは，中間層のユニット数`n_units`128，出力層のユニット数`out_units`を1とします．

各層のユニット数を`MLP`クラスの引数として与え，ネットワークを作成します．

学習を行う際の最適化方法としてSGD (確率的勾配降下法）を利用します．また，学習率を0.001として引数に与えます．そして，最適化方法のsetup関数にネットワークモデルを与えます．

In [ ]:
# ユニット数の定義
hidden_units = 128
out_units = 1

# ネットワークの作成
model = MLP(n_hidden=hidden_units, n_out=out_units)

# 最適化手法の設定
optimizer = chainer.optimizers.SGD(lr=0.001)
optimizer.setup(model)

## 学習
読み込んだbreast cancerデータセットと作成したネットワークを用いて，学習を行います．

1回の誤差を算出するデータ数（ミニバッチサイズ）を10，学習エポック数を200とします．

学習データは毎回ランダムに決定するため，numpyの`permutation`という関数を利用します．
各更新において，学習用データと教師データをそれぞれ`x`と`t`とします．
学習モデルに`x`を与えて各クラスの確率`y`を取得します．
各クラスの確率`y`と教師ラベル`t`との誤差を`sigmoid_coross_entropy`誤差関数で算出します．
また，認識精度も算出します．
そして，誤差を`backward`関数で逆伝播し，ネットワークの更新を行います．

In [ ]:
# ミニバッチサイズ・エポック数・学習データ数の設定
batch_size = 10
epoch_num = 200
train_data_num = x_train.shape[0]

# 学習の実行
for epoch in range(1, epoch_num + 1):
    
    sum_loss = 0.0
    sum_accuracy = 0.0
    
    perm = np.random.permutation(train_data_num)
    for i in range(0, train_data_num, batch_size):
        x = Variable(x_train[perm[i:i+batch_size]])
        t = Variable(y_train[perm[i:i+batch_size]].reshape(-1, 1))
        t_acc = Variable(y_train[perm[i:i+batch_size]])
        
        model.zerograds()
        y = model(x)

        # 精度の計算
        pred = F.concat((1 - F.sigmoid(y), F.sigmoid(y)), axis=1)
        acc = F.accuracy(pred, t_acc)
        
        loss = F.sigmoid_cross_entropy(y, t)
        loss.backward()
        optimizer.update()
        
        sum_loss += loss.data * batch_size
        sum_accuracy += acc.data * batch_size
        
    if epoch % 10 == 0:
        print("epoch: {}, mean loss: {}, mean accuracy: {}".format(epoch,
                                                                   sum_loss/train_data_num,
                                                                   sum_accuracy/train_data_num))

## テスト
学習したネットワークを用いて，テストデータに対する認識率の確認を行います．

In [ ]:
count = 0
test_data_num = x_test.shape[0]

with chainer.using_config('train', False), chainer.using_config('enable_backprop', False):

    for i in range(test_data_num):
        x = Variable(np.array([x_test[i]], dtype=np.float32))
        t = y_test[i]
        y = F.sigmoid(model(x))

        if y.data[0, 0] > 0.5:
            pred = 1
        else:
            pred = 0

        if pred == t:
            count += 1

print("test accuracy: {}".format(count / test_data_num))